In [1]:

# imports
import os
import sys
import types
import json

# figure size/format
fig_width = 10
fig_height = 5
fig_format = 'retina'
fig_dpi = 96

# matplotlib defaults / format
try:
  import matplotlib.pyplot as plt
  plt.rcParams['figure.figsize'] = (fig_width, fig_height)
  plt.rcParams['figure.dpi'] = fig_dpi
  plt.rcParams['savefig.dpi'] = fig_dpi
  from IPython.display import set_matplotlib_formats
  set_matplotlib_formats(fig_format)
except Exception:
  pass

# plotly use connected mode
try:
  import plotly.io as pio
  pio.renderers.default = "notebook_connected"
except Exception:
  pass

# enable pandas latex repr when targeting pdfs
try:
  import pandas as pd
  if fig_format == 'pdf':
    pd.set_option('display.latex.repr', True)
except Exception:
  pass



# output kernel dependencies
kernel_deps = dict()
for module in list(sys.modules.values()):
  # Some modules play games with sys.modules (e.g. email/__init__.py
  # in the standard library), and occasionally this can cause strange
  # failures in getattr.  Just ignore anything that's not an ordinary
  # module.
  if not isinstance(module, types.ModuleType):
    continue
  path = getattr(module, "__file__", None)
  if not path:
    continue
  if path.endswith(".pyc") or path.endswith(".pyo"):
    path = path[:-1]
  if not os.path.exists(path):
    continue
  kernel_deps[path] = os.stat(path).st_mtime
print(json.dumps(kernel_deps))

# set run_path if requested
if r'':
  os.chdir(r'')

# reset state
%reset

def ojs_define(**kwargs):
  import json
  from IPython.core.display import display, HTML

  # do some minor magic for convenience when handling pandas
  # dataframes
  def convert(v):
    try:
      import pandas as pd
    except ModuleNotFoundError: # don't do the magic when pandas is not available
      return v
    if type(v) == pd.DataFrame:
      j = json.loads(v.T.to_json(orient='split'))
      return dict((k,v) for (k,v) in zip(j["index"], j["data"]))
    else:
      return v
  
  v = dict(contents=list(dict(name=key, value=convert(value)) for (key, value) in kwargs.items()))
  display(HTML('<script type="ojs-define">' + json.dumps(v) + '</script>'), metadata=dict(ojs_define = True))
globals()["ojs_define"] = ojs_define


C:\Users\kerry\AppData\Local\Temp\ipykernel_11468\3609341226.py:20: DeprecationWarning:

`set_matplotlib_formats` is deprecated since IPython 7.23, directly use `matplotlib_inline.backend_inline.set_matplotlib_formats()`



{"C:\\Users\\kerry\\AppData\\Local\\Programs\\Python\\Python310\\lib\\importlib\\_bootstrap.py": 1647454514.0, "C:\\Users\\kerry\\AppData\\Local\\Programs\\Python\\Python310\\lib\\importlib\\_bootstrap_external.py": 1647454514.0, "C:\\Users\\kerry\\AppData\\Local\\Programs\\Python\\Python310\\lib\\codecs.py": 1647454514.0, "C:\\Users\\kerry\\AppData\\Local\\Programs\\Python\\Python310\\lib\\encodings\\aliases.py": 1647454514.0, "C:\\Users\\kerry\\AppData\\Local\\Programs\\Python\\Python310\\lib\\encodings\\__init__.py": 1647454514.0, "C:\\Users\\kerry\\AppData\\Local\\Programs\\Python\\Python310\\lib\\encodings\\utf_8.py": 1647454514.0, "C:\\Users\\kerry\\AppData\\Local\\Programs\\Python\\Python310\\lib\\encodings\\cp1252.py": 1647454514.0, "C:\\Users\\kerry\\AppData\\Local\\Programs\\Python\\Python310\\lib\\abc.py": 1647454514.0, "C:\\Users\\kerry\\AppData\\Local\\Programs\\Python\\Python310\\lib\\io.py": 1647454514.0, "C:\\Users\\kerry\\AppData\\Local\\Programs\\Python\\Python310\\li

In [2]:
import numpy as np
from scipy.stats import norm

def callBS(S, K, T, sigma, r, q=0):
    if S == 0:
        return 0
    else:
        d1 = np.log(S/K) + (r-q+0.5*sigma**2)*T
        d1 /= sigma*np.sqrt(T)
        d2 = d1 - sigma*np.sqrt(T)
        N1 = norm.cdf(d1)
        N2 = norm.cdf(d2)
        return np.exp(-q*T)*S*N1 - np.exp(-r*T)*K*N2


def callBS_delta(S, K, T, sigma, r, q=0):
    if S==0:
        return 0
    else:
        d1 = np.log(S/K) + (r-q+0.5*sigma**2)*T
        d1 /= sigma*np.sqrt(T)
        N1 = norm.cdf(d1)
        return np.exp(-q*T)*N1 

K = 50
T = 1
sigma = 0.3
r = 0.05

grid = np.linspace(0,101)
call = [callBS(x, K, T, sigma, r) for x in grid]
delta = callBS_delta(60, K, T, sigma, r)
lev = delta*60 - callBS(60, K, T, sigma, r)

import plotly.graph_objects as go

trace1 = go.Scatter(
  x = grid,
  y = call,
  mode="lines",
  name = "call",
  hovertemplate="stock price = $%{x:.2f}<br>call value = $%{y:.2f}<extra></extra>"
)

trace2 = go.Scatter(
  x = grid,
  y = [delta*x-lev for x in grid],
  mode="lines",
  name = "delta shares levered",
  hovertemplate="stock price = $%{x:.2f}<br>delta shares levered = $%{y:.2f}<extra></extra>"

)

fig = go.Figure()
fig.add_trace(trace1)
fig.add_trace(trace2)
fig.update_layout(
    xaxis_title="Stock Price",
    yaxis_title="",
    xaxis_title_font = {"size":18},
    template="plotly_white",
    xaxis_tickprefix="$",
    yaxis_tickprefix="$",
    height=600,  
    width=1000,
    legend=dict(
      x = 0.01,
      y = 0.99
    ),
    font_size=16
)
fig.show()

In [3]:
import numpy as np
from scipy.stats import norm

def putBS(S, K, T, sigma, r, q=0):
    if S == 0:
        return K
    else:
        d1 = np.log(S/K) + (r-q+0.5*sigma**2)*T
        d1 /= sigma*np.sqrt(T)
        d2 = d1 - sigma*np.sqrt(T)
        N1 = norm.cdf(-d1)
        N2 = norm.cdf(-d2)
        return np.exp(-r*T)*K*N2 - np.exp(-q*T)*S*N1


def putBS_delta(S, K, T, sigma, r, q=0):
    if S==0:
        return 0
    else:
        d1 = np.log(S/K) + (r-q+0.5*sigma**2)*T
        d1 /= sigma*np.sqrt(T)
        N1 = norm.cdf(-d1)
        return -np.exp(-q*T)*N1 

K = 50
T = 1
sigma = 0.3
r = 0.05

grid = np.linspace(0,101)
put = [putBS(x, K, T, sigma, r) for x in grid]
delta = putBS_delta(60, K, T, sigma, r)
cash = - delta*60 + putBS(60, K, T, sigma, r)

import plotly.graph_objects as go

trace1 = go.Scatter(
  x = grid,
  y = put,
  mode="lines",
  name = "call",
  hovertemplate="stock price = $%{x:.2f}<br>put value = $%{y:.2f}<extra></extra>"
)

trace2 = go.Scatter(
  x = grid,
  y = [delta*x-lev for x in grid],
  mode="lines",
  name = "cash and delta shares",
  hovertemplate="stock price = $%{x:.2f}<br>cash and delta shares = $%{y:.2f}<extra></extra>"

)

fig = go.Figure()
fig.add_trace(trace1)
fig.add_trace(trace2)
fig.update_layout(
    xaxis_title="Stock Price",
    yaxis_title="",
    xaxis_title_font = {"size":18},
    template="plotly_white",
    xaxis_tickprefix="$",
    yaxis_tickprefix="$",
    height=600,  
    width=1000,
    legend=dict(
      x = 0.01,
      y = 0.99
    ),
    font_size=16
)
fig.show()